# Case Study: Neuro

## Import Packages

In [15]:
%load_ext autoreload
%autoreload 2

import os
import sys
import glob
import pandas as pd
import math
import matplotlib.pyplot as plt
import datetime
# import from absolute directory
srcdir = '/Users/pengzhizhang/cellDancer/src'
srcdir = '/Users/shengyuli/Library/CloudStorage/OneDrive-HoustonMethodist/work/Velocity/bin/cellDancer_polish/src'
srcdir = '/Users/wanglab/Documents/ShengyuLi/Velocity/bin/celldancer_polish/src'
#srcdir = '/Users/chen.weiqing/Documents/GitHub/cellDancer/src'
sys.path.append(srcdir)
from celldancer import velocity_estimation
from celldancer.compute_cell_velocity import compute
from celldancer.pseudo_time import pseudo_time
import celldancer.cdplt as cdplt
from celldancer.cdplt import colormap

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Velocity Estimation for Sample Genes

In [16]:
%autoreload 2

raw_path="/Users/shengyuli/Library/CloudStorage/OneDrive-HoustonMethodist/work/Velocity/data/raw_data/DentateGyrus.csv"
raw_path='/Users/wanglab/Documents/ShengyuLi/Velocity/data/raw_data/DentateGyrus.csv'
#raw_path='/Users/chen.weiqing/code/celldancer/DentateGyrus.csv'
cell_type_u_s=pd.read_csv(raw_path)

In [17]:
cell_type_u_s

,gene_name,unsplice,splice,cellID,clusters,embedding1,embedding2
0,Rgs20,0.069478,0.021971,10X83_2:AAACGGGGTCTCTTTAx,ImmGranule2,18.931086,-1.862429
1,Rgs20,0.085834,0.016256,10X83_2:AACCATGGTTCAACCAx,ImmGranule2,18.419891,-1.282895
2,Rgs20,0.068644,0.047774,10X83_2:AACACGTTCTGAAAGAx,CA2-3-4,2.369887,16.868419
3,Rgs20,0.045387,0.018101,10X83_2:AAAGATGCATTGAGCTx,CA,-5.351040,10.676485
4,Rgs20,0.040457,0.012846,10X83_2:AACCATGTCTACTTACx,CA1-Sub,-6.189126,11.754900
...,...,...,...,...,...,...,...
39164255,Gpm6b,0.876650,1.276089,10X84_3:TTTCCTCCACCATCCTx,ImmGranule1,10.812611,-2.487668
39164256,Gpm6b,2.024897,5.152006,10X84_3:TTTGTCACATGAAGTAx,CA2-3-4,8.246204,23.482788
39164257,Gpm6b,1.848051,1.491445,10X84_3:TTTCCTCCACGGTAAGx,nIPC,-3.441272,-4.917364
39164258,Gpm6b,0.696361,1.189091,10X84_3:TTTGTCAAGCGTCAAGx,ImmGranule2,16.394199,-6.143549


In [20]:
raw_path='/Users/wanglab/Documents/ShengyuLi/Velocity/data/raw_data/mouse_endo_blood20to25_2000_genes_moment100.csv'
#raw_path='/Users/chen.weiqing/code/celldancer/DentateGyrus.csv'
cell_type_u_s=pd.read_csv(raw_path)
ordered=cell_type_u_s[['gene_name','u0','s0','cellID','clusters','embedding1','embedding2']]
ordered.rename(columns={'u0': 'unsplice', 's0': 'splice'}, inplace=True)
# renamed=ordered.rename(columns={'u0': 'unspliced', 's0': 'spliced'}, inplace=True)


In [21]:
ordered

,gene_name,unsplice,splice,cellID,clusters,embedding1,embedding2
0,Sox17,0.000000,0.043971,cell_363,Blood progenitors 2,3.460521,15.574629
1,Sox17,0.000000,0.000000,cell_382,Blood progenitors 2,2.490433,14.971734
2,Sox17,0.000000,0.018161,cell_385,Blood progenitors 2,2.351203,15.267069
3,Sox17,0.000000,0.000000,cell_393,Blood progenitors 2,5.899098,14.388825
4,Sox17,0.000000,0.000000,cell_398,Blood progenitors 2,4.823139,15.374831
...,...,...,...,...,...,...,...
24657995,Gm47283,0.214961,1.145533,cell_139318,Erythroid3,8.032358,7.603037
24657996,Gm47283,0.300111,1.072944,cell_139321,Erythroid3,10.352904,6.446736
24657997,Gm47283,0.292607,1.199875,cell_139326,Erythroid3,9.464873,7.261099
24657998,Gm47283,0.266031,1.114659,cell_139327,Erythroid3,9.990495,7.243880


In [22]:
ordered.to_csv('/Users/wanglab/Documents/ShengyuLi/Velocity/data/raw_data/GastrulationErythroid.csv',index=False)

In [18]:
%autoreload 2

#enu=1
n_jobs_list=[20]

for enu,n_jobs in enumerate(n_jobs_list):
    time_start=datetime.datetime.now()

    max_epoches=200
    check_val_every_n_epoch=10
    patience=3
    permutation_ratio=0.1
    speed_up=True
    norm_u_s=False
    norm_cell_distribution=True

    gene_list=['Ntrk2','Dctn3','Nav1','Psd3',\
               'Gnao1','Ncald','Klf7','Ank2',\
               'Evl','Cdh13','Ppfia2','Astn1',\
               'Nrxn1','Dcx','Prkcg','Pygb','Scn2a1',\
                'Tbc1d16','Slc1a3','Syt11']


    #gene_list=list(load_raw_data.gene_name.drop_duplicates())
    gene_list=None
    #result_path = '/Users/wanglab/Documents/ShengyuLi/Velocity/data/neuro/velocity_result/tunning_train'
    #result_path=None

    folder_name=("epoch"+str(max_epoches)+
                 "check_n"+str(check_val_every_n_epoch)+
                 "Ratio"+str(permutation_ratio)+
                 "autoD"+str(speed_up)+
                 "autoN"+str(norm_u_s)+
                 "p"+str(patience)+
                 "bin"+str(norm_cell_distribution)+
                'job'+str(n_jobs))

    save_path = '/Users/wanglab/Documents/ShengyuLi/Velocity/data/neuro/velocity_result/test_speed/'+folder_name+'/'

    if not os.path.isdir(save_path):
        os.mkdir(save_path)

    # Velocity Estimation
    loss_df, cellDancer_df=velocity_estimation.velocity(cell_type_u_s,\
                                                     gene_list=gene_list,\
                                                     max_epoches=max_epoches,\
                                                     check_val_every_n_epoch=check_val_every_n_epoch,\
                                                     patience=patience,\
                                                     permutation_ratio=permutation_ratio,\
                                                     speed_up=speed_up,\
                                                     norm_u_s=norm_u_s,\
                                                     norm_cell_distribution=norm_cell_distribution,\
                                                     n_jobs=n_jobs,\
                                                     save_path=save_path)
    time_end=datetime.datetime.now()
    time_spent=(time_end-time_start).seconds/60
    time_report=pd.DataFrame({'n_jobs':n_jobs,
                              'time_start':time_start,
                              'time_end':time_end,
                              'time_spent(min)':time_spent,
                              'max_epoches':max_epoches,
                              'check_val_every_n_epoch':check_val_every_n_epoch,
                              'permutation_ratio':permutation_ratio,
                              'speed_up':speed_up,
                              'norm_u_s':norm_u_s,
                              'patience':patience,
                              'norm_cell_distribution':norm_cell_distribution,
                             }, index=[enu])
    # if enu==0:
    time_report.to_csv('/Users/wanglab/Documents/ShengyuLi/Velocity/data/neuro/velocity_result/test_speed/test_speed_njob20_equal_intervel.csv',index=False)
    # else:
        #time_report.to_csv('/Users/wanglab/Documents/ShengyuLi/Velocity/data/neuro/velocity_result/test_speed/test_speed.csv',index=False,mode='a',header=False)



Using /Users/wanglab/Documents/ShengyuLi/Velocity/data/neuro/velocity_result/test_speed/epoch200check_n10Ratio0.1autoDTrueautoNFalsep3binTruejob20/cell_dancer_velocity_2022-05-27 10-35-37 as the output path.
Arranging genes for parallel job.
2159  genes were arranged to  108  portions.


Velocity Estimation:   9%|█▉                   | 10/108 [02:59<29:21, 17.98s/it]


KeyboardInterrupt: 

In [19]:
%autoreload 2

#enu=1
n_jobs_list=[40]

for enu,n_jobs in enumerate(n_jobs_list):
    time_start=datetime.datetime.now()

    max_epoches=200
    check_val_every_n_epoch=10
    patience=3
    permutation_ratio=0.1
    speed_up=True
    norm_u_s=False
    norm_cell_distribution=True

    gene_list=['Ntrk2','Dctn3','Nav1','Psd3',\
               'Gnao1','Ncald','Klf7','Ank2',\
               'Evl','Cdh13','Ppfia2','Astn1',\
               'Nrxn1','Dcx','Prkcg','Pygb','Scn2a1',\
                'Tbc1d16','Slc1a3','Syt11']


    #gene_list=list(load_raw_data.gene_name.drop_duplicates())
    gene_list=None
    #result_path = '/Users/wanglab/Documents/ShengyuLi/Velocity/data/neuro/velocity_result/tunning_train'
    #result_path=None

    folder_name=("epoch"+str(max_epoches)+
                 "check_n"+str(check_val_every_n_epoch)+
                 "Ratio"+str(permutation_ratio)+
                 "autoD"+str(speed_up)+
                 "autoN"+str(norm_u_s)+
                 "p"+str(patience)+
                 "bin"+str(norm_cell_distribution)+
                'job'+str(n_jobs))

    save_path = '/Users/wanglab/Documents/ShengyuLi/Velocity/data/neuro/velocity_result/test_speed/'+folder_name+'/'

    if not os.path.isdir(save_path):
        os.mkdir(save_path)

    # Velocity Estimation
    loss_df, cellDancer_df=velocity_estimation.velocity(cell_type_u_s,\
                                                     gene_list=gene_list,\
                                                     max_epoches=max_epoches,\
                                                     check_val_every_n_epoch=check_val_every_n_epoch,\
                                                     patience=patience,\
                                                     permutation_ratio=permutation_ratio,\
                                                     speed_up=speed_up,\
                                                     norm_u_s=norm_u_s,\
                                                     norm_cell_distribution=norm_cell_distribution,\
                                                     n_jobs=n_jobs,\
                                                     save_path=save_path)
    time_end=datetime.datetime.now()
    time_spent=(time_end-time_start).seconds/60
    time_report=pd.DataFrame({'n_jobs':n_jobs,
                              'time_start':time_start,
                              'time_end':time_end,
                              'time_spent(min)':time_spent,
                              'max_epoches':max_epoches,
                              'check_val_every_n_epoch':check_val_every_n_epoch,
                              'permutation_ratio':permutation_ratio,
                              'speed_up':speed_up,
                              'norm_u_s':norm_u_s,
                              'patience':patience,
                              'norm_cell_distribution':norm_cell_distribution,
                             }, index=[enu])
    # if enu==0:
    time_report.to_csv('/Users/wanglab/Documents/ShengyuLi/Velocity/data/neuro/velocity_result/test_speed/test_speed_njob40_equal_intervel.csv',index=False)
    # else:
        #time_report.to_csv('/Users/wanglab/Documents/ShengyuLi/Velocity/data/neuro/velocity_result/test_speed/test_speed.csv',index=False,mode='a',header=False)




Using /Users/wanglab/Documents/ShengyuLi/Velocity/data/neuro/velocity_result/test_speed/epoch200check_n10Ratio0.1autoDTrueautoNFalsep3binTruejob40/cell_dancer_velocity_2022-05-27 10-38-58 as the output path.
Arranging genes for parallel job.
2159  genes were arranged to  54  portions.


Velocity Estimation:  11%|██▌                    | 6/54 [03:47<30:19, 37.91s/it]


KeyboardInterrupt: 

In [ ]:
%autoreload 2

#enu=1
n_jobs_list=[40,35,30,25,20,15,10,5,1]

for enu,n_jobs in enumerate(n_jobs_list):
    time_start=datetime.datetime.now()

    max_epoches=200
    check_val_every_n_epoch=10
    patience=3
    permutation_ratio=0.1
    speed_up=True
    norm_u_s=False
    norm_cell_distribution=True

    gene_list=['Ntrk2','Dctn3','Nav1','Psd3',\
               'Gnao1','Ncald','Klf7','Ank2',\
               'Evl','Cdh13','Ppfia2','Astn1',\
               'Nrxn1','Dcx','Prkcg','Pygb','Scn2a1',\
                'Tbc1d16','Slc1a3','Syt11']


    #gene_list=list(load_raw_data.gene_name.drop_duplicates())
    gene_list=None
    #result_path = '/Users/wanglab/Documents/ShengyuLi/Velocity/data/neuro/velocity_result/tunning_train'
    #result_path=None

    folder_name=("epoch"+str(max_epoches)+
                 "check_n"+str(check_val_every_n_epoch)+
                 "Ratio"+str(permutation_ratio)+
                 "autoD"+str(speed_up)+
                 "autoN"+str(norm_u_s)+
                 "p"+str(patience)+
                 "bin"+str(norm_cell_distribution)+
                'job'+str(n_jobs))

    save_path = '/Users/wanglab/Documents/ShengyuLi/Velocity/data/neuro/velocity_result/test_speed/'+folder_name+'/'

    if not os.path.isdir(save_path):
        os.mkdir(save_path)

    # Velocity Estimation
    loss_df, cellDancer_df=velocity_estimation.velocity(cell_type_u_s,\
                                                     gene_list=gene_list,\
                                                     max_epoches=max_epoches,\
                                                     check_val_every_n_epoch=check_val_every_n_epoch,\
                                                     patience=patience,\
                                                     permutation_ratio=permutation_ratio,\
                                                     speed_up=speed_up,\
                                                     norm_u_s=norm_u_s,\
                                                     norm_cell_distribution=norm_cell_distribution,\
                                                     n_jobs=n_jobs,\
                                                     save_path=save_path)
    time_end=datetime.datetime.now()
    time_spent=(time_end-time_start).seconds/60
    time_report=pd.DataFrame({'n_jobs':n_jobs,
                              'time_start':time_start,
                              'time_end':time_end,
                              'time_spent(min)':time_spent,
                              'max_epoches':max_epoches,
                              'check_val_every_n_epoch':check_val_every_n_epoch,
                              'permutation_ratio':permutation_ratio,
                              'speed_up':speed_up,
                              'norm_u_s':norm_u_s,
                              'patience':patience,
                              'norm_cell_distribution':norm_cell_distribution,
                             }, index=[enu])
    # if enu==0:
    time_report.to_csv('/Users/wanglab/Documents/ShengyuLi/Velocity/data/neuro/velocity_result/test_speed/test_speed_equal_intervel.csv',index=False)
    # else:
        #time_report.to_csv('/Users/wanglab/Documents/ShengyuLi/Velocity/data/neuro/velocity_result/test_speed/test_speed.csv',index=False,mode='a',header=False)


